In [81]:
import igraph as ig
import numpy as np

def updated_attack(graph, attack, out=None, random_state=0):
    
    ## Set random seed for reproducibility
    np.random.seed(random_state)

    ## Create a copy of graph so as not to modify the original
    g = graph.copy()

    ## Save original index as a vertex property
    N0 = g.vcount()
    g.vs['name'] = range(N0)
    
    ## List with the node original indices in removal order
    original_indices = []

    j = 0
    if out:
        if os.path.isfile(out) and os.path.getsize(out) > 0: 
            oi_values = np.loadtxt(out, dtype='int', comments='\x00')
            g.delete_vertices(oi_values)
            oi_values = np.array(oi_values) ## In case oi_values is one single integer
            j += len(oi_values)
            np.savetxt(out, oi_values, fmt='%d')

        f = open(out, 'a+')

    while j < N0:

        ## Identify node to be removed
        if attack == 'BtwU':
            c_values = g.betweenness(directed=False, nobigint=False)
        elif attack == 'DegU':
            c_values = g.degree()
        idx = np.argmax(c_values)

        ## Add index to list
        original_idx = g.vs[idx]['name']
        original_indices.append(original_idx)

        ## Remove node
        g.vs[idx].delete()     

        j += 1
            
        if out:
            f.write('{:d}\n'.format(original_idx))
            f.flush()
        
    if out:
        f.close()

    return original_indices

def DegU_attack(graph, random_state=0):
    
    ## Set random seed for reproducibility
    np.random.seed(random_state)

    ## Create a copy of graph so as not to modify the original
    g = graph.copy()
    
    degrees = g.degree()
    
    ## Save original index as a vertex property
    N0 = g.vcount()
    
    ## Neighbors list
    neighbors = [[] for i in range(N)]
    for i, v in enumerate(g.vs()):
        neighbors[i] = set([])
        neighbors[i] = set([w.index for w in v.neighbors()])

    ## List with the node original indices in removal order
    original_indices = []

    j = 0
    while j < N0:

        ## Identify node to be removed
        v = np.argmax(degrees)

        ##Remove node
        degrees[v] = 0
        for w in neighbors[v]:
            degrees[w] -= 1      

        ## Add index to list
        original_indices.append(v)  

        j += 1
        
    return original_indices

In [87]:
N = 4000
p = 0.00025
g = ig.Graph().Erdos_Renyi(N, p)
g.summary()

'IGRAPH U--- 4000 1986 -- '

In [74]:
%timeit updated_attack(g, 'DegU')

177 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
%timeit DegU_attack(g)

KeyboardInterrupt: 

In [88]:
oi_list = updated_attack(g, 'DegU')
oi_list[:10]

[194, 400, 559, 1289, 1544, 1715, 1890, 2078, 2536, 2571]

In [89]:
oi_list_new = DegU_attack(g)
oi_list_new[:10]

[194, 400, 559, 1289, 1544, 1715, 1890, 2078, 2536, 2571]

In [98]:
N = 8000
p = 0.000125
graph = ig.Graph().Erdos_Renyi(N, p)
graph.summary()

'IGRAPH U--- 8000 4106 -- '

In [107]:
#g = graph.copy()
N0 = g.vcount()
neighbors = [[] for i in range(N0)]
for i, v in enumerate(g.vs()):
    neighbors[i] = set([])
    neighbors[i] = set([w.index for w in v.neighbors()])    
degrees = g.degree()

#deg_oi_values = {d: oi for oi, d in enumerate(degrees)}

#oi_values = np.argsort(degrees)[::-1]
#degrees = np.sort(degrees)[::-1]

## List with the node original indices in removal order
original_indices = []

d_prev = np.max(degrees)

j = 0
while j < N0:

    ## Identify node to be removed
    #v = np.argmax(degrees)
    
    for oi, d in degrees:
        if d >= max_d:
            max_d = d
            v = oi
        if d >= d_prev:
            max_d = d
            break
        
    #oi = deg_oi_values
    #v = j
    ##Remove node
    degrees[v] = 0
    for w in neighbors[v]:
        degrees[w] -= 1      

    ## Add index to list
    original_indices.append(v)  

    j += 1

original_indices[:10]

[617, 1517, 1906, 26, 48, 136, 244, 364, 1797, 1937]

In [109]:
#g = graph.copy()
N0 = g.vcount()
neighbors = [[] for i in range(N0)]
for i, v in enumerate(g.vs()):
    neighbors[i] = set([])
    neighbors[i] = set([w.index for w in v.neighbors()])    

In [110]:
oi_values = np.argsort(degrees)
degrees = np.sort(degrees)

In [115]:
for oi, d in list(zip(oi_values, degrees))[:10]:
    print(oi, d, g.vs[oi].degree())

0 0 0
1244 0 0
1242 0 0
3275 0 0
1235 0 0
2403 0 0
1233 0 0
1231 0 0
2405 0 0
2406 0 0


In [122]:
def_diff = np.where(np.diff(degrees))[0]

In [124]:
v = oi_values[-1]
for w in neighbors[v]:
    

{176, 707, 2074, 3348, 3834, 3887}